# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [22]:
#Lendo o arquivo excel e extraindo os tweets e suas respectivas classificaçoes, sendo 0 = irrelevante e 1 = relevante.
data = pd.read_excel("mcdonalds_tweets.xlsx",sheet_name = "Treinamento")
colunas = ["Treinamento", "Classificação"]
data = data.loc[:,colunas]


#Limpando caracteres irrelevantes:
#https://pt.stackoverflow.com/questions/217832/como-retirar-caractere-especial-e-ponto-de-coluna-string-de-um-data-frame

def deletar_caracteres(coluna):
    coluna = coluna.replace("."," ").replace(":"," ").replace(","," ").replace("'","").replace('"', " ").replace("#"," ")\
    .replace("-","")
    return coluna



    

data['Treinamento'] = data['Treinamento'].apply(deletar_caracteres)





In [23]:
#Dividindo entre relevante e irrelevante e criando um dataframe com os dados
Relevante = data[data["Classificação"] == 1]
irrel = data[data["Classificação"] == 0] 

dic_palavras_rel = {}
dic_palavras_irrel = {}

for i in Relevante["Treinamento"]:
    x = i.split()
    for e in x:
        if e not in dic_palavras_rel:
            dic_palavras_rel[e] = 1
        
        elif e in dic_palavras_rel:
            dic_palavras_rel[e] += 1



            
for i in irrel["Treinamento"]:
    x = i.split()
    for e in x:
        if e not in dic_palavras_irrel:
            dic_palavras_irrel[e] = 1
        
        elif e in dic_palavras_irrel:
            dic_palavras_irrel[e] += 1
            

dic_palavras_re = pd.DataFrame.from_dict(dic_palavras_rel, orient='index')
dic_palavras_irre =  pd.DataFrame.from_dict(dic_palavras_irrel,  orient='index')


dic_palavras_re.columns = ["Relevante"]
dic_palavras_irre.columns = ["Irrelevante"]


Dataframe =  dic_palavras_re.join(dic_palavras_irre, how='outer')


Dataframe = Dataframe.fillna(0)
Dataframe.reset_index(level=0, inplace=True)
Dataframe.rename(columns = {"index":'Palavras'}, inplace=True)

Dataframe['Relevante'] = Dataframe['Relevante'].astype(int)
Dataframe['Irrelevante'] = Dataframe['Irrelevante'].astype(int)







In [24]:
# Total de palavras relevantes e irrelevantes

quantidade_rel = Dataframe.Relevante.sum()

quantidade_irrel = Dataframe.Irrelevante.sum() 

total_palavras = quantidade_rel + quantidade_irrel





        



In [25]:
#Probabilidade de palavra/relevante e palavra/irrelevante


Dataframe = Dataframe.copy()

Dataframe['prob_rel'] = (Dataframe.Relevante + 1) / (quantidade_rel+ total_palavras)
Dataframe['prob_irrel'] = (Dataframe.Irrelevante + 1) / (quantidade_irrel+ total_palavras)



In [26]:
# Probabilidade do tweet ser relevante ou irrelevante

total_1 = data['Treinamento'][data['Classificação'] == 1].count()
total_2 = data['Treinamento'][data['Classificação'] == 0].count()

Prelevante = total_1/(total_1+total_2)
Pirrelevante = total_2/(total_1+total_2)

print(Prelevante)
print(Pirrelevante)




0.59
0.41


In [27]:
#Probabilidade de relevante/palavra e irrelevante/palavra
lis1 = []
lis2 = []
for e in Dataframe.prob_rel:
    lis1.append(e*Prelevante)
for e in Dataframe.prob_irrel:
    lis2.append(e*Pirrelevante)


    
Prob = {}    

Prob2 = {}
        
        
for i in range(0,len(Dataframe.Relevante)):
    Prob[Dataframe.Palavras[i]] = lis1[i]
    

    
for i in range(0,len(Dataframe.Irrelevante)):
    Prob2[Dataframe.Palavras[i]] = lis2[i]
        
index = pd.RangeIndex(start=0, stop=1, step=1)  

    
dc = pd.DataFrame(data=Prob,index=index)
x=dc.T
dc0 = pd.DataFrame(data=Prob2,index=index)
z=dc0.T
x.columns = ["Relevante_Palavra"]
z.columns = ["Irrelevante_Palavra"]

Dataframe_Prob =  x.join(z, how='inner')


Dataframe



        

        
            


 


,Palavras,Relevante,Irrelevante,prob_rel,prob_irrel
0,&amp;,0,1,0.000138,0.000297
1,&gt;,0,8,0.000138,0.001337
2,(@,2,0,0.000415,0.000149
3,(sabado,1,0,0.000277,0.000149
4,//t,37,65,0.005257,0.009807
5,000,1,0,0.000277,0.000149
6,01,1,0,0.000277,0.000149
7,1,1,0,0.000277,0.000149
8,10,1,1,0.000277,0.000297
9,123m,0,1,0.000138,0.000297


In [28]:
# Criando uma funçao para determinar se o tweet é relevante ou irrelevante dado as informações anteriores 
data_teste = pd.read_excel("mcdonalds_tweets.xlsx",sheet_name = "Teste")
colunas = ["Teste", "Classificação"]
data_teste = data_teste.loc[:,colunas] 
data_teste['Teste'] = data_teste['Teste'].apply(deletar_caracteres)

lis_teste = []
for i in data_teste["Teste"]:
    lis_teste.append(i)
    


def classificar (tweet):
    counter_rel = 1
    counter_irrel = 1
    palavras = tweet.split()
    for e in range(len(palavras)):
        palavra = palavras[e]
        if palavra in Dataframe_Prob.index:
            counter_rel *= Prob.get(palavra)
            counter_irrel *= Prob2.get(palavra)
        if palavra not in Dataframe_Prob.index:
            counter_rel *= (0 + 1) / (quantidade_rel+ total_palavras)
            counter_irrel *= (0 + 1) / (quantidade_irrel+ total_palavras)
            
    if counter_rel > counter_irrel:
        return 1
       
        
    else:
        return 0


lista  = []      
for e in lis_teste:
    lista.append(classificar(e))    
    


    
data_teste['Classificados_Bayes'] =  lista


                
   



        
        
data_teste
    

    
    
    

,Teste,Classificação,Classificados_Bayes
0,rt @ricardooliv1704 burger king é bem melhor ...,1,1
1,era só um lanche do mcdonalds,1,1
2,rt @awannable younghoon tentando falar mcdon...,0,0
3,rt @mcdonalds_br finalmente é quartafeira di...,0,1
4,eu diria que é muito fácil amar alguém mais do...,1,1
5,tortinha do mcdonalds é uma decepção tô chora...,1,1
6,só eu que achei nada a ver ter ovo no sanduích...,1,1
7,rt @veja golpe do mcdonald’s promete dois com...,1,1
8,rt @fhjxregui só para dizer que o mcdonalds d...,1,1
9,gostei de um vídeo @youtube https //t co/jzbia...,0,1


In [29]:
# Descobrindo quantos tweets foram classificados como relevante e irrelevante pelo teorema de bayes e quantos foram\
# previamentes marcados no teste .

relevante_teste = data_teste.Classificação.sum()
relevante_bayes = data_teste.Classificados_Bayes.sum()

irrelevante_teste = 200 - relevante_teste
irrelevante_bayes = 200 - relevante_bayes



            
    

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [30]:
print("Relevantes no teste : {0} tweets".format(relevante_teste))
print("Relevantes no Bayes : {0} tweets".format(relevante_bayes))
print("irrelevantes no teste : {0} tweets".format(irrelevante_teste))
print("irrelevantes no Bayes : {0} tweets".format(irrelevante_bayes))

positivos_falsos = 0
positivos_verdadeiros = 0 
negativos_verdadeiros = 0
negativos_falsos = 0

for frase,valor in zip(data_teste["Teste"], data_teste["Classificação"]):
    bayes = classificar(frase)
    if bayes == 1 and int(valor) == 1 :
        positivos_verdadeiros +=1
    if bayes == 1 and int(valor) == 0:
        positivos_falsos += 1
    if bayes == 0 and int(valor) == 0:
        negativos_verdadeiros +=1
    if bayes ==0 and int(valor) ==1:
        negativos_falsos +=1
        
pf = 100*positivos_falsos/200
pv = 100*positivos_verdadeiros/200
nv = 100*negativos_verdadeiros/200
nf = 100*negativos_falsos/200
porcentagem = pv +nv

print("Porcentagem de Acerto: {0:.2f}%".format(porcentagem))
print('Porcentagem de positivos falsos',pf,'%')
print('Porcentagem de positivos verdadeiros',pv,'%')
print('Porcentagem de negativos verdadeiros',nv,'%')
print('Porcentagem de negativos falsos',nf,'%')


        
        
        
    

Relevantes no teste : 127 tweets
Relevantes no Bayes : 158 tweets
irrelevantes no teste : 73 tweets
irrelevantes no Bayes : 42 tweets
Porcentagem de Acerto: 80.50%
Porcentagem de positivos falsos 17.5 %
Porcentagem de positivos verdadeiros 61.5 %
Porcentagem de negativos verdadeiros 19.0 %
Porcentagem de negativos falsos 2.0 %


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


    Analizando as medidas obtidas, nota-se que a porcentagem de positivos e negativos verdadeiros, são maiores do que os positivos e negativos falsos, percebe-se também que com a probabilidade padrão de ser relevante de 59 % contra a acurácia de 80,5% obtida pelo algoritimo, o codigo foi eficaz na classificação da relevância dos tweets sobre o McDonald's.
    Mensagens de negação e sarcasmo foram tratadas como relevantes, uma vez que elas se referiam, idependentemente de ser positiva ou negativa, ao McDonald's.
    Este algorítimo não pode ser usado para melhorar a sua acurácia, devido ao acúmulo de erro. Se sua taxa de acerto inicial é de 80%, usando ele mesmo para aumentar a base de treinamento iria fazer com que sua acurácia decaísse (80% x 80% = 64%).
    Esse projeto deve continuar a ser financiado devido ao fato de poder ser aplicado em diferentes tipos de pesquisas. Por exemplo, qual a probabilidade de uma pessoa votar em um certo candidato dado que ela mora em São Paulo, qual a probabiblidade de uma pessoa comprar um ar condicionado dado que ela mora na Bahia ou qual a probabilidade de uma pessoa ter um barco dado que ela mora no Amazonas. Este algoritimo é muito poderoso e é capaz de classificar basicamente qualquer evento binário com uma alta acurácia e de forma quase intantânea. Por isso é tão fundamental seu uso neste tipo de pesquisas.
    Entretanto, o algoritimo desconsidera a correlação entre as variáveis, para tornar esse classificador ainda mais eficiente seria nescessario implementar medidas para solucionar esse problema, alguns pesquisadores ja tentam fazer isso através do relaxamento da hipotese de independencia condicional entre os atributos. Outra melhoria para o classificador nesse caso seria desconsiderar os retweets, pois o retweet faz com que a mesma frase apareça diversas vezes fazendo com que a probabilidade de uma mesma palavra aumente, dessa maneira, cria um falso resultado da probabilidade da palavra, prejudicando a acurácia do classificador.
    
    
    


    
   
    